In [ ]:
!pip install -U spacy
# Kleines Modell: de_core_news_sm
# Größeres Modell: de_core_news_md
!python -m spacy download de_core_news_sm
import spacy
from collections import defaultdict

# Deutsches Sprachmodell laden
nlp = spacy.load("de_core_news_sm")  # Kleines deutsches Modell

def build_ngram_model_with_spacy(text, n):
    # Text mit SpaCy verarbeiten
    doc = nlp(text)

    # Tokens extrahieren und filtern
    tokens = [token.text.lower() for token in doc
              if not token.is_punct and not token.is_space]

    # N-Gramm-Zählungen initialisieren
    ngram_counts = defaultdict(lambda: defaultdict(int))

    # Durch alle möglichen N-Gramme iterieren
    for i in range(len(tokens) - n + 1):
        # Die ersten n-1 Wörter sind der Kontext
        context = tuple(tokens[i:i+n-1])
        next_word = tokens[i+n-1]

        # Mit defaultdict brauchen wir keine Existenzprüfung mehr
        ngram_counts[context][next_word] += 1

    return ngram_counts


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 96.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import requests
import tarfile
import os

url = "https://downloads.wortschatz-leipzig.de/corpora/deu_mixed-typical_2011_10K.tar.gz"
filename = "deu_mixed-typical_2011_10K.tar.gz"
extracted_dir = "deu_mixed-typical_2011_10K"
text_file_name = "deu_mixed-typical_2011_10K-sentences.txt" # Annahme basierend auf typischen Formaten

# 1. Datei herunterladen
print(f"Lade Datei von {url} herunter...")
response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(filename, 'wb') as f:
        f.write(response.raw.read())
    print("Download abgeschlossen.")
else:
    print(f"Fehler beim Herunterladen: Statuscode {response.status_code}")

# 2. Datei entpacken
print(f"Entpacke Datei {filename}...")
if os.path.exists(filename):
    with tarfile.open(filename, "r:gz") as tar:
        tar.extractall(".")
    print("Entpacken abgeschlossen.")
else:
    print(f"Fehler: Datei {filename} nicht gefunden.")

# 3. Text extrahieren
print(f"Extrahiere Text aus {text_file_name}...")
full_text = ""
file_path = os.path.join(extracted_dir, text_file_name)

if os.path.exists(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # Zeilenaufbau: Zeilennummer + Tab + Text
            parts = line.strip().split('\t', 1)
            if len(parts) > 1:
                full_text += parts[1] + " " # Füge den Text und ein Leerzeichen hinzu
    print("Textextraktion abgeschlossen.")
else:
    print(f"Fehler: Textdatei {file_path} nicht gefunden. Bitte prüfen Sie den Dateinamen im Tar-Archiv.")


# 4. N-Gramm-Modell generieren (nutzt die vorhandene Funktion)
if full_text:
    print("Generiere N-Gramm-Modell...")
    # Verwenden Sie ein 3-Gramm-Modell (Kontext von 2 Wörtern)
    ngram_model = build_ngram_model_with_spacy(full_text, n=3)
    print("Modellgenerierung abgeschlossen.")

    # 5. Auto-Complete Funktion
    def autocomplete(model, prompt, n=3):
        # Verarbeite den Prompt mit SpaCy
        doc = nlp(prompt)
        # Extrahiere die letzten n-1 Tokens des Prompts
        # Stelle sicher, dass wir mindestens n-1 Tokens haben, bevor wir slicen
        prompt_tokens = [token.text.lower() for token in doc
                         if not token.is_punct and not token.is_space]

        # Wenn der Prompt weniger als n-1 Tokens hat, können wir keine Vorhersage treffen
        if len(prompt_tokens) < n - 1:
            return []

        context = tuple(prompt_tokens[-(n-1):]) # Die letzten n-1 Tokens bilden den Kontext

        # Überprüfe, ob der Kontext im Modell vorhanden ist
        if context in model:
            # Hole dir die möglichen nächsten Wörter und ihre Zählungen
            next_words_counts = model[context]
            # Sortiere die Vorschläge nach Häufigkeit (absteigend)
            suggestions = sorted(next_words_counts.items(), key=lambda item: item[1], reverse=True)
            # Gib die vorgeschlagenen Wörter zurück
            return [word for word, count in suggestions]
        else:
            # Wenn der Kontext nicht im Modell gefunden wird, gib eine leere Liste zurück
            return []

    # Beispiel für die Verwendung des Auto-Complete
    print("\nAuto-Complete Beispiel:")
    prompt1 = "Er wurde im"
    suggestions1 = autocomplete(ngram_model, prompt1, n=3)
    print(f"Prompt: '{prompt1}'")
    print(f"Vorschläge: {suggestions1[:5]}") # Zeige die Top 5 Vorschläge

    prompt2 = "Es gibt sie"
    suggestions2 = autocomplete(ngram_model, prompt2, n=3)
    print(f"\nPrompt: '{prompt2}'")
    print(f"Vorschläge: {suggestions2[:5]}") # Zeige die Top 5 Vorschläge

    prompt3 = "Dieses Wort kommt nicht"
    suggestions3 = autocomplete(ngram_model, prompt3, n=3)
    print(f"\nPrompt: '{prompt3}'")
    print(f"Vorschläge: {suggestions3[:5]}") # Zeige die Top 5 Vorschläge (sollte leer sein, wenn Kontext unbekannt)


else:
    print("Kein Text zum Modellieren gefunden. Auto-Complete nicht verfügbar.")


Lade Datei von https://downloads.wortschatz-leipzig.de/corpora/deu_mixed-typical_2011_10K.tar.gz herunter...
Download abgeschlossen.
Entpacke Datei deu_mixed-typical_2011_10K.tar.gz...
Entpacken abgeschlossen.
Extrahiere Text aus deu_mixed-typical_2011_10K-sentences.txt...
Textextraktion abgeschlossen.
Generiere N-Gramm-Modell...
Modellgenerierung abgeschlossen.

Auto-Complete Beispiel:
Prompt: 'Er wurde im'
Vorschläge: ['umkreis', 'schuljahr', '19.', 'prozeß', 'jahre']

Prompt: 'Es gibt sie'
Vorschläge: ['an', 'in']

Prompt: 'Dieses Wort kommt nicht'
Vorschläge: []


In [ ]:
!python -m spacy download de_core_news_md
nlp = spacy.load("de_core_news_md")  # Kleines deutsches Modell
# NER-Beispiel mit SpaCy
print("\nNER-Beispiel mit SpaCy:")
# Wähle einen kurzen Textausschnitt aus dem heruntergeladenen Corpus
sample_text = full_text[:500] # Die ersten 500 Zeichen als Beispiel

# Verarbeite den Beispieltext mit SpaCy
doc = nlp(sample_text)

# Extrahiere benannte Entitäten (Named Entities)
print("Benannte Entitäten im Beispieltext:")
for ent in doc.ents:
    print(f"  Text: {ent.text}, Typ: {ent.label_}, Erklärung: {spacy.explain(ent.label_)}")

print("\nNER-Beispiel mit SpaCy:")
# Wähle einen kurzen Textausschnitt aus dem heruntergeladenen Corpus
sample_text = "Schrödinger kann nicht nur Programmieren. Er kennt sich jetzt auch mit dem Thema KI gut aus."

# Verarbeite den Beispieltext mit SpaCy
doc = nlp(sample_text)

# Extrahiere benannte Entitäten (Named Entities)
print("Benannte Entitäten im Beispieltext:")
for ent in doc.ents:
  print(f"  Text: {ent.text}, Typ: {ent.label_}, Erklärung: {spacy.explain(ent.label_)}")
else:
  print("Kein Text für das NER-Beispiel gefunden.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 MB 19.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.

NER-Beispiel mit SpaCy:
Benannte Entitäten im Beispieltext:
  Text: Zentralturm, Typ: LOC, Erklärung: Non-GPE locations, mountain ranges, bodies of water
  Text: München, Typ: LOC, Erklärung: Non-GPE locations, mountain ranges, bodies of water
  Text: Mariazell, Typ: LOC, Erklärung: Non-GPE locations, mountain ranges, bodies of water
  Text: Deštnice, Typ: LOC, Erklärung: Non-GPE locations, mountain ranges, bodies of water
  Text: SIRA-Bataillon, Typ: MISC, Erklärung: Miscellaneous entities, e.g. events, nationalities, products or works of art
  Text: Hannover, Typ: LOC, Erk